# Prerequisites

Before running the code in this notebook, please ensure that you have the following packages installed. You can install them using the provided commands:

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install opencv-python-headless
!pip install Pillow
!pip install keras
!pip install tensorboard

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
from keras.models import load_model

Download training dataset:
https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Zip file will contain "Train" folder, this should be moved to same directory as this code.

In [ ]:
#Code to check your current working directory:
import os

print("Current working directory:", os.getcwd())

#This is where the "Train" folder should be located

# Prepare Dataset

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

#Train folder should be in your working directory.
DATADIR='Train' # Setting the directory of the dataset
class_dict = {0: 'Speed limit (20km/h)', #links name of folder to its category
              1: 'Speed limit (30km/h)', 
              2: 'Speed limit (50km/h)', 
              3: 'Speed limit (60km/h)', 
              4: 'Speed limit (70km/h)', 
              5: 'Speed limit (80km/h)', 
              6: 'End of speed limit (80km/h)', 
              7: 'Speed limit (100km/h)', 
              8: 'Speed limit (120km/h)', 
              9: 'No passing', 
              10: 'No passing veh over 3.5 tons', 
              11: 'Right-of-way at intersection', 
              12: 'Priority road', 
              13: 'Yield', 
              14: 'Stop', 
              15: 'No vehicles', 
              16: 'Veh > 3.5 tons prohibited', 
              17: 'No entry', 
              18: 'General caution', 
              19: 'Dangerous curve left', 
              20: 'Dangerous curve right', 
              21: 'Double curve', 
              22: 'Bumpy road', 
              23: 'Slippery road', 
              24: 'Road narrows on the right', 
              25: 'Road work', 
              26: 'Traffic signals', 
              27: 'Pedestrians', 
              28: 'Children crossing', 
              29: 'Bicycles crossing', 
              30: 'Beware of ice/snow', 
              31: 'Wild animals crossing', 
              32: 'End speed + passing limits', 
              33: 'Turn right ahead', 
              34: 'Turn left ahead', 
              35: 'Ahead only', 
              36: 'Go straight or right', 
              37: 'Go straight or left', 
              38: 'Keep right', 
              39: 'Keep left', 
              40: 'Roundabout mandatory', 
              41: 'End of no passing', 
              42: 'End no passing vehicle with a weight greater than 3.5 tons'} 

# Loop through each category folder in the class_dict dictionary
for index, category in class_dict.items():
    # Create a path to the current category folder by joining the main directory (DATADIR) with the folder index (converted to a string)
    path = os.path.join(DATADIR, str(index))  
    # Loop through each image file in the current category folder
    for img in os.listdir(path):
        # Read the image file using OpenCV, with the flag cv2.IMREAD_COLOR to ensure it's read in color format
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)     
        # Display the image using matplotlib
        plt.imshow(img_array)        
        # Add a title to the displayed image with the category name
        plt.title(category)        
        # Show the image plot
        plt.show()        
        break
        
    break


In [ ]:
print(img_array.shape) #images not the same shape

In [ ]:
IMG_SIZE=50

new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)) #makes all of the images 50x50
plt.imshow(new_array)
plt.show()

# Assign Training Data

In [ ]:
training_data = []

def create_training_data(): #Function to assign training data
    for index, category in class_dict.items(): #Iterate through the dictionary of categories
        path = os.path.join(DATADIR, str(index)) #Goes into each category folder
        class_num = index #assigns a number to the current category
        for img in os.listdir(path): # Loops through each image in current folder
            try:
                # Read the current image file, stores the image array in img_array
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                
                # Resizes image to 50x50
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE), cv2.IMREAD_COLOR)
                
                # Image array stored in list, with class_num as its label
                training_data.append([new_array, class_num])
            except Exception as e:
                  pass

create_training_data()


In [ ]:
print(len(training_data)) #39209 total images 

In [ ]:
import random #shuffles the training data
random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]: #goes through 10 samples in training data
    print(class_dict[sample[1]]) #prints the category of each sample

In [ ]:
X=[]
y=[]

#loops through the training data, 
#assinging the features/lables to the X and Y labels
for features,label in training_data: 
    X.append(features)
    y.append(label)
    
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE, 3) #the 3 means 3 color chanels
#reshaped to 50x50

y=np.array(y)

# Export/Save Training/Testing Data

In [ ]:
import pickle

pickle_out=open('X.pickle','wb')
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out=open('y.pickle','wb')
pickle.dump(y,pickle_out)
pickle_out.close()

# Import Data to use in model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

NAME = "trafficsignCNN32-{}".format(int(time.time()))

tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

#Loads in features and labels
X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))

#Normalizes pixel values from 0-1 (rather than 1-256)
X = X/255.0

model = Sequential()

#First convolution layer (128 filters)
model.add(Conv2D(128, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Second Convolution layer (128 filters)
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Activation('relu'))

#Output layer, set to 43, one for each category
model.add(Dense(43))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#If the ETA is greater than 5 minutes, you may want to consider only doing 1 epoch
model.fit(X, y, batch_size=32, epochs=5, validation_split=0.20, callbacks=[tensorboard])

#Name that model will be saved as
model.save('traffic_model')

Accuracy should be above 80%, if it is under 10%, there is likely an issue.

# Model Testing Prerequisites:

Using any external images of a single traffic sign type, save them to a folder in the working directory, this folder will be used as an input for the testing code below, so you may want to name it something that you will remember/understand, such as "stopsign_test". 
Please follow comments in code below to test your images.

# Tests external images

In [ ]:
import cv2
import tensorflow as tf
import os

def prepare(filepath):
    IMG_SIZE=50 #This should match the image size used in the training set
    img_array=cv2.imread(filepath)
    new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)

#Loads in model that was created
model=tf.keras.models.load_model('traffic_model')

#Sets folder to be tested that is in your directory
test_folder = 'stopsign_test' #set this to the name of your testing folder
correct_predictions = 0
total_predictions = 0

#Loops through all the images
for filename in os.listdir(test_folder):
    if filename.endswith(".png"):  # Selects images that are .png files
        file_path = os.path.join(test_folder, filename)
        prediction = model.predict([prepare(file_path)]) #makes a prediction
        predicted_category = class_dict[prediction[0].argmax()] #uses the prediction to select a category
        
        print(f"File: {filename}, Predicted Category: {predicted_category}") #prints out each prediction
         
        # Set this to the category you are predicting, this name should match 
        # a class_dict name, such as "yield" or "stop"
        if predicted_category == "Stop": 
            correct_predictions += 1
        total_predictions += 1

#Accuracy is printed based on number of correct predictions
accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy: {accuracy}%")


# GUI Code

This section runs the graphical user interface.

In [ ]:
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

#load the trained model to classify sign
from keras.models import load_model
from tensorflow.keras.preprocessing import image

class_dict = {0: 'Speed limit (20km/h)', #links name of folder to its category
              1: 'Speed limit (30km/h)', 
              2: 'Speed limit (50km/h)', 
              3: 'Speed limit (60km/h)', 
              4: 'Speed limit (70km/h)', 
              5: 'Speed limit (80km/h)', 
              6: 'End of speed limit (80km/h)', 
              7: 'Speed limit (100km/h)', 
              8: 'Speed limit (120km/h)', 
              9: 'No passing', 
              10: 'No passing veh over 3.5 tons', 
              11: 'Right-of-way at intersection', 
              12: 'Priority road', 
              13: 'Yield', 
              14: 'Stop', 
              15: 'No vehicles', 
              16: 'Veh > 3.5 tons prohibited', 
              17: 'No entry', 
              18: 'General caution', 
              19: 'Dangerous curve left', 
              20: 'Dangerous curve right', 
              21: 'Double curve', 
              22: 'Bumpy road', 
              23: 'Slippery road', 
              24: 'Road narrows on the right', 
              25: 'Road work', 
              26: 'Traffic signals', 
              27: 'Pedestrians', 
              28: 'Children crossing', 
              29: 'Bicycles crossing', 
              30: 'Beware of ice/snow', 
              31: 'Wild animals crossing', 
              32: 'End speed + passing limits', 
              33: 'Turn right ahead', 
              34: 'Turn left ahead', 
              35: 'Ahead only', 
              36: 'Go straight or right', 
              37: 'Go straight or left', 
              38: 'Keep right', 
              39: 'Keep left', 
              40: 'Roundabout mandatory', 
              41: 'End of no passing', 
              42: 'End no passing vehicle with a weight greater than 3.5 tons'} 

#Load model created earlier
model = load_model('traffic_model')
#initialise GUI
top=tk.Tk()
top.geometry('800x600')
top.title('Traffic sign classification')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)

import cv2

def prepare_image(file_path):
    IMG_SIZE = 50 #This should also match the training data image size
    img_array = cv2.imread(file_path)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)


def classify(file_path):
    global label_packed
    img_array = prepare_image(file_path)
    pred = model.predict(img_array)[0]
    sign = class_dict[pred.argmax()]
    print(sign)
    label.configure(foreground='#011638', text=sign)


def show_classify_button(file_path):
    classify_b = Button(top, text="Classify Image", command=lambda: classify(file_path), padx=10, pady=5)
    classify_b.configure(background='#364156', foreground='white', font=('arial', 10, 'bold'))
    classify_b.place(relx=0.79, rely=0.46)


def upload_image():
    try:
        file_path = filedialog.askopenfilename()
        uploaded = Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width() / 2.25), (top.winfo_height() / 2.25)))
        im = ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image = im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass


upload = Button(top, text="Upload an image", command=upload_image, padx=10, pady=5)
upload.configure(background='#364156', foreground='white', font=('arial', 10, 'bold'))
upload.pack(side=BOTTOM, pady=50)
sign_image.pack(side=BOTTOM, expand=True)
label.pack(side=BOTTOM, expand=True)
heading = Label(top, text="Check traffic sign", pady=20, font=('arial', 20, 'bold'))
heading.configure(background='#CDCDCD', foreground='#364156')
heading.pack()
top.mainloop()

Optional: Code optimization via tensorboard

The following section of code allows you to test different combinations of dense layers, convolution layers, and number of neurons in each layer, the instructions for testing these out is commented in the code below.

Viewing results:
In terminal/cmd, go to the tensorboard file location (For example: C:\Users\t0112np\Anaconda3\Scripts). This location may vary depending on where your files are stored. 
Next, type "tensorboard --logdir=" followed up by the logs file location created in your working directory. Here is an example:    C:\Users\t0112np\Desktop\Python_final_Project\logs

This should provide a localhost link, copy and paste this into your browseer, this will show information on the model.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 30 12:59:19 2023

@author: T0112NP
"""

import tensorflow as tf   # Import TensorFlow library
from tensorflow.keras.datasets import cifar10   # Import CIFAR-10 dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator   # Import image data generator from Keras
from tensorflow.keras.models import Sequential   # Import Sequential model from Keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten   # Import Dense, Dropout, Activation, and Flatten layers from Keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D   # Import Conv2D and MaxPooling2D layers from Keras
from tensorflow.keras.callbacks import TensorBoard   # Import TensorBoard callback from Keras

# Define a name for the model for TensorBoard logging purposes
NAME = "Traffic-Signals-CNN-128x128x64"

import pickle   # Import pickle module for loading data from pickle files
import time   # Import time module for generating a unique name for the model

# Load the preprocessed images and labels from pickle files
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

# Normalize the pixel values of the images to the range [0,1]
X = X/255.0

# Define the hyperparameters to iterate over for different model configurations
dense_layers = [1,2,3]#you can create a list to go over different values[1,3,5..]
layer_sizes = [128]#you can create a list to go over different values[64,128,256..]
conv_layers = [2]#you can create a list to go over different values[1,2,3,4,5..]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            # Generate a unique name for the model using the current hyperparameters and the current timestamp
            # NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            # print(NAME)
            
            # Define the CNN architecture
            model = Sequential()
            
            model.add(Conv2D(128, (3, 3), input_shape=X.shape[1:]))   # Add the first convolutional layer with 128 filters of size 3x3 and the input shape of the images
            model.add(Activation('relu'))   # Add the ReLU activation function
            model.add(MaxPooling2D(pool_size=(2, 2)))   # Add the max pooling layer with pool size of 2x2(window size to reduce the value of the image size)
            
            # Add additional convolutional layers with the specified number of filters and the ReLU activation function
            for l in range(conv_layer-1):#conv_layer-1, where the -1 refers to the 1st hidden layer included with the image input size
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
            
            model.add(Flatten())   # Flatten the output of the convolutional layers to a 1D vector
            
            # Add dense layers(pre-Output Layer) with the specified number of nodes and the ReLU activation function
            for d in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
            
            # Add the output layer with 43 nodes (one for each class) and the softmax activation function
            model.add(Dense(43))
            model.add(Activation('softmax'))
            
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))   # Define a TensorBoard callback with the log directory based on the model name
            
            model.compile(loss='sparse_categorical_crossentropy',   # Compile the model with the sparse categorical cross-entropy loss function
                          optimizer='adam',   # Use the Adam optimizer
                          metrics=['accuracy'])   # Evaluate the model's accuracy
            
            # Train the model for one epoch on a batch size
            model.fit(X, y, batch_size=64, epochs=3, validation_split=0.3, callbacks=[tensorboard])
model.save('Traffic-Signals-CNN-ximg_64')